In [5]:
import os
import yaml
import json
import pandas as pd

In [6]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
%autoreload 2

In [8]:
import wombat.bsub as bsub

make test input yaml

In [9]:
template = {
    'specimen_id': 'HT206B1-S1H5_U1_07012023_test',
    'input_tif': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/sandbox/test_images/HT206B1-S1H5_TOP_HT297B1-S1H1_HT339B2-H2_HT565B1-S1H2_BTTM_Scan1.qptiff'
    },
    'bbox': '4000,6000,7000,9000',
    'platform': 'phenocycler',
    'nuclei_channels': 'DAPI',
    'membrane_channels': 'Pan-Cytokeratin,E-cadherin,CD45,CD8,CD3e,CD4,Vimentin,SMA,CD31',
    'mask_markers': 'Pan-Cytokeratin,E-cadherin',
    'group': 'HTAN',
    'project': 'Multiplex_Imaging'
}

In [11]:
yaml.safe_dump(template, open('../tests/data/full_imaging_workflow/inputs.yaml', 'w'))

In [18]:
template = {
    'specimen_id': 'HT206B1-S1H5_U1_07012023_test',
    'input_tif': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/sandbox/test_images/HT206B1-S1H5_TOP_HT297B1-S1H1_HT339B2-H2_HT565B1-S1H2_BTTM_Scan1.qptiff'
    },
    'bbox': '0,10000,3000,13000',
    'platform': 'phenocycler',
    'nuclei_channels': 'DAPI',
    'membrane_channels': 'Pan-Cytokeratin,E-cadherin,CD45,CD8,CD3e,CD4,Vimentin,SMA,CD31',
    'mask_markers': 'Pan-Cytokeratin,E-cadherin',
    'group': 'HTAN',
    'project': 'Multiplex_Imaging'
}

In [19]:
yaml.safe_dump(template, open('../tests/data/full_imaging_workflow/inputs_full.yaml', 'w'))

make cromwell compute1 config

In [12]:
workflow_root = '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/full_imaging_workflow'
output_fp = '../tests/data/full_imaging_workflow/full_imaging_workflow.cromwell-config-db.compute1.dat'
bsub.save_compute1_cromwell_template(workflow_root, output_fp)

In [13]:
# !mkdir -p ../tests/data/full_imaging_workflow

make execution script

In [14]:
TEST_DIR_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/full_imaging_workflow'
TOOL_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/multiplex-imaging-pipeline'

dconfig = os.path.join(TEST_DIR_ROOT, 'full_imaging_workflow.cromwell-config-db.compute1.dat')
cwl_fp = os.path.join(TOOL_ROOT, 'cwl', 'full_imaging_workflow.cwl')
inputs_fp = os.path.join(TEST_DIR_ROOT, 'inputs.yaml')
volumes = [TEST_DIR_ROOT, TOOL_ROOT, workflow_root, '/storage1/fs1/dinglab', '/scratch1/fs1/dinglab']


In [15]:
args = bsub.DEFAULT_ARGS
args['group_name'] = 'test_cromwell'
args

{'mem': 10,
 'n_processes': 1,
 'max_mem': None,
 'docker': 'python:3.8',
 'queue': 'dinglab',
 'gpu_model': 'TeslaV100_SXM2_32GB',
 'gpu_mem': '30',
 'gpu_num': 1,
 'use_gpu': False,
 'group': 'compute-dinglab',
 'group_name': 'test_cromwell',
 'n_concurrent': 10,
 'interactive': False,
 'username': 'estorrs'}

In [16]:
run_commands = bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, args, volumes, workflow_root=workflow_root, log_fp='log.txt')
filepath = os.path.join('../tests/data/full_imaging_workflow/run_job.sh')
bsub.write_command_file(run_commands, filepath)

In [17]:
for c in run_commands: print(c)

mkdir -p /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/full_imaging_workflow/cromwell-workdir/logs
source /opt/ibm/lsfsuite/lsf/conf/lsf.conf
export LSF_DOCKER_NETWORK=host
export LSF_DOCKER_VOLUMES="/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/full_imaging_workflow:/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/full_imaging_workflow /storage1/fs1/dinglab/Active/Projects/estorrs/multiplex-imaging-pipeline:/storage1/fs1/dinglab/Active/Projects/estorrs/multiplex-imaging-pipeline /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/full_imaging_workflow:/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/full_imaging_workflow /storage1/fs1/dinglab:/storage1/fs1/dinglab /scratch1/fs1/dinglab:/scratch1/fs1/dinglab"
bgadd -L 10 /estorrs/test_cromwell
export PATH="/opt/java/openjdk/bin:$PATH"
bsub -R 'select[mem>10GB] rusage[mem=10GB] span[hosts=1]' -M 11GB -n 1 -q dinglab -G compute-dinglab -a 'docker(estorrs/cromwell-runner:58)' -g 

In [20]:
inputs_fp = os.path.join(TEST_DIR_ROOT, 'inputs.yaml')
run_commands = bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, args, volumes, workflow_root=workflow_root, log_fp='log.txt')
filepath = os.path.join('../tests/data/full_imaging_workflow/run_job_full.sh')
bsub.write_command_file(run_commands, filepath)